In [1]:
import pandas as pd
import io, gzip
from codecs import decode
import pysrt
import datetime as dt
import base64
import re
from tqdm import tqdm
from os import listdir
import pdb
import dill
from datetime import datetime
import numpy as np
import sys
from xmlrpc.client import ServerProxy
import os as os
import os.path
import json
import time


In [2]:
with open('credentials.pkd','rb') as fp:
    credentials = dill.load(fp)
USER_AGENT=credentials['user']
USERNAME=credentials['user']
PASS=credentials['passw']

In [3]:
OPENSUBTITLES_SERVER = 'http://api.opensubtitles.org/xml-rpc'
LANGUAGE = 'en'

xmlrpc = ServerProxy(OPENSUBTITLES_SERVER,allow_none=True)
data = xmlrpc.LogIn(USERNAME, PASS, LANGUAGE, USER_AGENT)
status=data.get('status')
if status != '200 OK':
    print('Could not log in. Something is wrong... ')
    print("data.get('status')")
    token = None
else:
    token = data.get('token')
    print('Logged in. Acquired token = {}'.format(token))
def server_stat():
    print('------------------Current Stats-----------------')
    for key,item in xmlrpc.ServerInfo(token)['download_limits'].items():
        print('{}: {}'.format(key,item))
    return
def get_down_quota():
    return xmlrpc.ServerInfo(token)['download_limits']['client_download_quota']

Logged in. Acquired token = zg98trTSQ0Z2JemuRWixHLGQX77


In [4]:
server_stat()

------------------Current Stats-----------------
global_24h_download_limit: 200
client_ip: 71.183.36.65
limit_check_by: user_id
user_id: 5937493
client_24h_download_count: 1000
client_download_quota: 0
client_24h_download_limit: 1000


In [5]:
with open('movie_DB_1_7_2018.pkd', 'rb') as fp:
    movie_database = dill.load(fp)

# Step 1 - Subtitle_ID's
Using out database of IMDB_ID's we seek 3 Subtitle_ID's in order of quality. Best one is first. We will then attempt download of the best, and if that doesnt work, the next one and so on... We do the search in batches of 500 whichis the maximum allowed.


In [6]:
def filter_one(sr_one):
    if not sr_one:
        return ()
    else:
        try:
            sr=[(item['SubDownloadsCnt'],item['SubRating'],item['IDSubtitleFile'],item['IDMovieImdb']) for item in sr_one 
             if item['SubFormat']=='srt' if not re.findall('.*[^a-zA-Z]cd[^a-zA-Z]',item['MovieReleaseName'].lower())]
            sr_sort=sorted(sr, key=lambda x: -int(x[0]))
            result=tuple([fill_id(sr_sort[0][3])]+[item[2] for item in sr_sort[0:3]])
            #pdb.set_trace()
            return result #returns a tuple 
        except:
            return ()

def fill_id(string):
    #opensubtitles removes the 'tt' and all leading 0's . this function replaces them.
    while len(string) < 7:
        string = '0' + string
    return 'tt' + string
    
def search_imdbid(list_imdbids):
    #SearchSubtitles is crappy because it also gives you 500 search results
    #useless to search for 500 movies at a time since each one might have ~100 results
    #for safety i will search for 10 at a time ~ 2000 requests ~ 40per 10seconds ~ 10-15 mins maybe 
    search_params=[{'sublanguageid':'eng','imdbid':item.strip('tt')} for item in list_imdbids]
    search_data=xmlrpc.SearchSubtitles(token,search_params)
    sr=[]
    try_num=0
    while sr==[]:
        if search_data['status'] == '200 OK':
            sd=search_data['data'] #a long list of search results by imdb_ids
            uniq_id=list(set([item['IDMovieImdb'] for item in sd]))
            sr=[filter_one([item for item in sd if item['IDMovieImdb'] == idnum]) for idnum in uniq_id] 
            #filter_one takes a list of search results for a single imdbid and returns a tuple (imdbid, subid1,subid2,subid3)
            #the above 3 lines will partition the search results sd into a list of lists where each inner list is for one imdbid
            if sr==[]: #sr will be an empty list if no results are found (it is possible)
                return []         
        else:
            if try_num==3: #gives up after 3 times
                print('Tried 3 times. Giving up.')
                return 
            try_num=try_num+1
            print('Error Code: {} Retrying in 10 seconds.'.format(search_data['status']))    
            time.sleep(10)
    return sr

# Take movie_database-queries opebsubs-returns sub_id list 

In [ ]:
#LA=list of imdb_ids to get

In [7]:
t0 = time.time()
sub_ids=[]
size=10
for i in tqdm(range(0,len(LA),size)):
    sub_ids=sub_ids+search_imdbid(LA[i:i+size])
print('\nElapsed {} seconds.'.format(time.time()-t0))    

NameError: name 'LA' is not defined

Clean the subs by removing all empty tuples which exist because some subs cannot be found. 
**And write to file.**

In [267]:
sub_id_clean=[item for item in sub_ids if item!=()]
with open('sub_id_temp.pkd', 'wb') as fp:
    dill.dump(sub_id_clean, fp)

In [8]:
with open('sub_id_clean.pkd', 'rb') as fp:
    sub_id_clean = dill.load(fp)
print('Number of sub_ids {}'.format(len(sub_id_clean)))

Number of sub_ids 16893


# Misc Functions

In [9]:
def get_title(imdb_id):
    try:
        ind=[item['imdb_id'] for item in movie_database].index(imdb_id)
        return movie_database[ind]['title'] 
    except:
        return 'Not Found.'

# Downloading the subs

In [10]:
def down_sub(sub_id):
    downsub=xmlrpc.DownloadSubtitles(token, [sub_id])
    if downsub['status'] != '200 OK':
        print('Error Code: {}'.format(downsub['status']))
        server_stat()
        exit('Leaving this world now.')
    return downsub

def decode_sub(enc_subfile):
    dec_data=decode(bytearray(enc_subfile['data'][0]['data'].encode("utf-8")),'base64')
    try:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("utf-8")       
    except:
        sub_txt=gzip.GzipFile(fileobj=io.BytesIO(dec_data)).read().decode("ISO-8859-1")   
    return sub_txt

def write_to_file(file_name, file_content):
    #writes file_content into an srt file in the subs_storage subfolder of current path
    #if exists prints message
    #if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ file_name + '.srt'):
    #    print('Sub '+ file_name + ' exists already.')
    #else:
    with open(os.getcwd()+'\\subs_storage\\'+ file_name +'.srt','wb') as fp:
        fp.write(file_content.encode('utf-8'))

# THIS PART ACTIVELY DOWNLOADS
Checks to see what *.srt files exist in the folder. 
1. If they are there nothing happens.
2. If they are not there then it uses sub_id_clean file to get the best sub listing. 

<img src="http://clipartix.com/wp-content/uploads/2017/06/Stop-sign-clip-art-microsoft-free-clipart-images-2.png" alt="Le bag of words" title="The bag of words" />

**RUNNING THE NEXT CELL WILL CAUSE DOWNLOADING TO OCCUR. YOU ONLY HAVE 1000 PER DAY.**
**MAKE SURE EVERYTHING IS GOOD BEFORE RUNINNG**

In [591]:
for item in sub_id_clean[6]:
    if os.path.isfile(os.getcwd()+'\\subs_storage\\'+ item[0] + '.srt'):
        print('SUB_ID:{:<10} IMDB_ID:{} exists ---- SKIPPING FILE Title: {}'.format(item[1],item[0],get_title(item[0])))
    else:
        print('SUB_ID:{:<10} IMDB_ID:{} downloading ------------- Title: {}'.format(item[1],item[0],get_title(item[0])))
        write_to_file(item[0],decode_sub(down_sub(item[1])))

SUB_ID:t          IMDB_ID:t downloading ------------- Title: Not Found.
Error Code: 407 Download limit reached
------------------Current Stats-----------------
global_24h_download_limit: 200
client_ip: 71.183.36.65
limit_check_by: user_id
user_id: 5937493
client_24h_download_count: 1000
client_download_quota: 0
client_24h_download_limit: 1000


SystemExit: Leaving this world now.

C:\Users\Adi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Parsing SRT Files

In [34]:
work_dir=os.getcwd()+'\\subs_storage\\'
def read_srt(imdb_id):
    if os.path.isfile(work_dir+imdb_id + '.srt'):
        with open(work_dir+imdb_id+'.srt','rb') as fp:
            test=fp.read().decode('utf-8')
        return test
    else:
        pass

def fparse1(string):
    if string:
        string=re.sub(r'<i>','',string)
        string=re.sub(r'</i>','',string)
        string=re.sub(r'\ufeff','',string)
        string=re.sub('^[^(1)]*','',string) #removes stuff before the 1
        return string
    else:
        pass

def srt_conv(subs):
    if subs:
        starttimes=[]
        endtimes=[]
        words=[]
        indexes=[]
        #"-->" in line
        subs=io.StringIO(subs)
        prevLineWasWords =False
        prevLineWasTime =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
                prevLineWasTime =True
            elif line.strip().isdigit() and not prevLineWasTime:
                indexes.append(int(line))
            elif len(line) > 1:
                    if prevLineWasWords:
                            words[len(words)-1] = words[len(words)-1] + " " + line
                    else:
                            words.append(line)
                    prevLineWasWords = True
                    continue
            prevLineWasTime = False
            prevLineWasWords = False  
        return starttimes, endtimes, words,indexes
    else:
        pass
    
def watch_output(clean_srt):
    subs=io.StringIO(clean_srt)
    print('=== is number, ~~~ is text, --- is time index. \n')
    for line in subs:
        if "-->" in line:
            print('----'+line)
        elif line.strip().isdigit():
            print('====='+line)
        elif len(line)>1:
            print('~~~~'+line)
    return

def s_time(string):
    return datetime.strptime(string, '%H:%M:%S')

def make_df(imdb_id):
    srt1=fparse1(read_srt(imdb_id))
    if srt1:
        a,b,c,_=srt_conv(srt1)

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        data_sub=pd.DataFrame(columns=index)


        s1 = pd.Series([s_time(item) for item in a])
        s2 = pd.Series([s_time(item) for item in b])
        s3 = pd.Series(c)
        data_sub.loc[:,(imdb_id,'StartTime')] =s1.values
        data_sub.loc[:,(imdb_id,'EndTime')] = s2.values
        data_sub.loc[:,(imdb_id,'Line')] = s3.values

        return data_sub
    else:
        pass
    
def append_pd(df1,df):
    tes=pd.concat([df1,df], axis=1)
    tes=tes.fillna('')
    return tes

In [35]:
aid=[item.split('.')[0] for item in listdir(work_dir) if 'tt' in item]

In [36]:
full_data=pd.DataFrame()
for i in range(100):
    print('adding {}'.format(aid[i]))
    full_data=append_pd(full_data,make_df(aid[i]))

adding tt0004284
adding tt0006177
adding tt0007162
adding tt0007264
adding tt0007832
adding tt0011841
adding tt0012255
adding tt0021735
adding tt0022698
adding tt0022883
adding tt0023194
adding tt0023876
adding tt0024991
adding tt0025200
adding tt0025919
adding tt0026164
adding tt0026205
adding tt0027376
adding tt0027442
adding tt0027657
adding tt0027893
adding tt0028203
adding tt0028315
adding tt0028981
adding tt0029146
adding tt0029161
adding tt0029217
adding tt0029648
adding tt0029864
adding tt0029929
adding tt0029930
adding tt0030944
adding tt0031088
adding tt0031241
adding tt0031359
adding tt0031377
adding tt0031507
adding tt0032143
adding tt0032194
adding tt0032234
adding tt0032390
adding tt0032520
adding tt0033253
adding tt0033388
adding tt0033627
adding tt0033874
adding tt0034399
adding tt0034798
adding tt0034922
adding tt0035317


ValueError: Must have equal len keys and value when setting with an iterable

In [92]:
test=(fparse1(read_srt('tt0035317')))

In [110]:
subs=test
starttimes=[]
endtimes=[]
words=[]
indexes=[]
#"-->" in line
subs=io.StringIO(subs)
prevLineWasWords =False
prevLineWasTime =False
for line in subs:
    line=line.strip()
    if "-->" in line:
        starttimes.append(line[:8])
        endtimes.append(line[17:25])
        prevLineWasTime=True
    elif line.strip().isdigit() and not prevLineWasTime:
        indexes.append(int(line))
    elif len(line) > 1:
            if prevLineWasWords:
                    words[len(words)-1] = words[len(words)-1] + " " + line
            else:
                    words.append(line)
            prevLineWasWords = True
            continue
    prevLineWasWords = False
    prevLineWasTime=False
return starttimes, endtimes, words, indexes



SyntaxError: 'return' outside function (<ipython-input-110-16de838fda10>, line 27)

In [104]:
'601'.isdigit()

True

In [39]:
a,b,c,_=srt_conv(test)

In [106]:
watch_output(test[-2003:])

=== is number, ~~~ is text, --- is time index. 

=====1394

----01:03:40,923 --> 01:03:42,483

~~~~I have a trap set,

~~~~

=====1395

----01:03:42,563 --> 01:03:44,163

~~~~electric eye principle.

~~~~

=====1396

----01:03:44,203 --> 01:03:46,243

~~~~My passing through

~~~~will break the beam

~~~~

=====1397

----01:03:46,283 --> 01:03:48,083

~~~~and automatically open

~~~~the highly deceptive

~~~~

=====1398

----01:03:48,163 --> 01:03:51,723

~~~~trap door behind

~~~~me, of course.

~~~~

=====1399

----01:03:51,803 --> 01:03:54,643

~~~~My pursuer, meaning

~~~~you my dear Holmes,

~~~~

=====1400

----01:03:54,723 --> 01:03:56,003

~~~~will then be plunged

~~~~

=====1401

----01:03:56,043 --> 01:03:57,683

~~~~sixty feet into

~~~~the sewers below.

~~~~

=====1402

----01:03:57,763 --> 01:03:59,083

~~~~Holmes.

~~~~

=====1403

----01:04:04,963 --> 01:04:07,403

~~~~Holmes.

~~~~

=====1404

----01:04:07,483 --> 01:04:09,443

~~~~Poor Moriarity.

~~~~

=====1405

---

In [89]:
pp=zip(a[0:1423],b[0:1423],c[0:1423])

In [90]:
for item in pp:
    print(item)

('00:01:59', '00:02:02', "Shh, he's here.")
('00:02:15', '00:02:17', 'You gentlemen look like')
('00:02:17', '00:02:18', 'educated men of good taste.')
('00:02:19', '00:02:22', 'I have here some rare old first editions.')
('00:02:22', '00:02:23', "He's not bothering us.")
('00:02:23', '00:02:26', "I'm interested in his books.")
('00:02:26', '00:02:28', 'Okay.')
('00:02:28', '00:02:32', 'Now here are the complete works')
('00:02:32', '00:02:34', 'of William Shakespeare,')
('00:02:34', '00:02:37', 'an old German writer.')
('00:02:37', '00:02:38', 'I like that.')
('00:02:38', '00:02:40', 'We move tonight.')
('00:02:40', '00:02:41', 'Orders from Berlin.')
('00:02:41', '00:02:42', 'Dr. Tobel is to be')
('00:02:42', '00:02:44', 'across the border before dawn.')
('00:02:44', '00:02:45', "But we'd had orders not to break into his house")
('00:02:46', '00:02:47', 'and he hides there.')
('00:02:47', '00:02:49', "Hasn't been outside in weeks.")
('00:02:49', '00:02:51', 'The Fuhrer wants no troubl

('00:10:01', '00:10:03', 'is where everyone can see it.')
('00:10:03', '00:10:04', 'Yes, but...')
('00:10:05', '00:10:06', 'You will remember no doubt')
('00:10:06', '00:10:10', 'in Poes story "The Purloined Letter".')
('00:10:10', '00:10:12', 'And the missing in question')
('00:10:12', '00:10:15', 'was always in plain view.')
('00:10:20', '00:10:22', 'Hands up, gentlemen.')
('00:10:31', '00:10:34', 'Scotland Yard, quick.')
('00:10:34', '00:10:36', 'Oh, my goodness.')
('00:10:37', '00:10:39', 'Good evening, Mrs. Hudson.')
('00:10:39', '00:10:42', "Oh, why it's Mr. Holmes.")
('00:10:42', '00:10:43', 'Holmes!')
('00:10:43', '00:10:44', 'Hello Watson old fellow,')
('00:10:44', '00:10:46', "it's good to see you again.")
('00:10:46', '00:10:47', 'Telephone.')
('00:10:47', '00:10:48', 'Huh?')
('00:10:49', '00:10:50', 'Who do you want?')
('00:10:50', '00:10:52', "Oh yes, you're Scotland Yard.")
('00:10:52', '00:10:54', "I'm afraid there's been a little mistake.")
('00:10:54', '00:10:55', 'No 

('00:16:25', '00:16:28', 'than you could possibly realize, Charlotte.')
('00:16:31', '00:16:32', 'Good.')
('00:16:36', '00:16:38', 'More coffee?')
('00:16:38', '00:16:40', 'No, thank you, darling.')
('00:16:40', '00:16:42', 'I must get back before they miss me.')
('00:16:42', '00:16:44', "We've been separated for so long.")
('00:16:44', '00:16:45', "I couldn't bear")
('00:16:45', '00:16:47', 'that anything should part us again.')
('00:16:47', '00:16:48', 'I want to work with you')
('00:16:48', '00:16:51', 'and I want to know every minute where you are.')
('00:16:51', '00:16:52', 'Even for you')
('00:16:52', '00:16:54', 'to know the details of my mission in London')
('00:16:54', '00:16:56', 'is to sign your death warrant.')
('00:16:56', '00:16:59', 'If you are in real danger I want to share it.')
('00:16:59', '00:17:02', 'There is one thing you can do.')
('00:17:05', '00:17:07', 'Guard this envelope.')
('00:17:07', '00:17:08', 'If anything happens to me')
('00:17:08', '00:17:10', 'see t

('00:23:20', '00:23:22', 'It was excellent')
('00:23:22', '00:23:23', 'amazing performance.')
('00:23:23', '00:23:25', 'It was magnificent, Dr. Tobel.')
('00:23:25', '00:23:27', 'Magnificent.')
('00:23:27', '00:23:28', 'I am glad we had such')
('00:23:28', '00:23:30', 'ideal conditions for the test.')
('00:23:30', '00:23:31', "I wouldn't have believed it")
('00:23:31', '00:23:32', "if I hadn't seen it myself.")
('00:23:32', '00:23:33', "We've been through a lot, Holmes")
('00:23:33', '00:23:35', "but thank goodness he's safe now.")
('00:23:35', '00:23:38', 'You brought your apparatus with you?')
('00:23:38', '00:23:39', 'No, Sir Reginald.')
('00:23:40', '00:23:41', 'I did not.')
('00:23:41', '00:23:43', 'But why not?')
('00:23:43', '00:23:44', 'It must be lodged')
('00:23:44', '00:23:46', 'in the most impregnable vault in this building')
('00:23:46', '00:23:47', "until we're ready to start manufacture.")
('00:23:47', '00:23:50', 'I have arranged for a little office of my own.')
('00:23

('00:28:13', '00:28:14', 'Sherlock Holmes calling.')
('00:28:15', '00:28:17', 'So Holmes is finally stumped.')
('00:28:17', '00:28:18', "First time I've heard him call for help.")
('00:28:18', '00:28:20', 'Call for help indeed.')
('00:28:20', '00:28:22', "He's only just beginning his investigation.")
('00:28:22', '00:28:23', 'Lestrade?')
('00:28:23', '00:28:24', 'This is Holmes.')
('00:28:24', '00:28:26', 'Check the records and place a call')
('00:28:26', '00:28:28', 'made on my private telephone of Bakers Street')
('00:28:28', '00:28:29', "approximately twelve o'clock")
('00:28:29', '00:28:31', "on the night of Dr. Tobel's arrival in London.")
('00:28:31', '00:28:33', 'Finally stumped, hey Sir Reginald.')
('00:28:33', '00:28:34', 'Yes, Lestrade?')
('00:28:34', '00:28:38', 'Wait a minute.')
('00:28:38', '00:28:40', 'Charlotte Eberli,')
('00:28:41', '00:28:42', "St. George's Street,")
('00:28:42', '00:28:44', 'Flat B.')
('00:28:44', '00:28:46', 'Excellent, Lestrade.')
('00:28:46', '00:2

('00:34:44', '00:34:46', "Well it ain't so much to ask,")
('00:34:46', '00:34:48', 'for what you want to know.')
('00:34:48', '00:34:51', 'You make good bargain.')
('00:34:51', '00:34:54', 'That does it.')
('00:35:00', '00:35:02', 'Well?')
('00:35:02', '00:35:04', "You remember Angel's Court.")
('00:35:04', '00:35:05', 'Huh.')
('00:35:06', '00:35:07', 'Then follow your nose through the alley')
('00:35:07', '00:35:09', "till you come to Jed Brady's Carpenter Shop.")
('00:35:09', '00:35:11', 'He can tell you where the blighter is')
('00:35:11', '00:35:13', 'at this very moment.')
('00:35:13', '00:35:15', 'You come too.')
('00:35:16', '00:35:16', 'Me?')
('00:35:17', '00:35:19', 'If you lie.')
('00:35:19', '00:35:20', 'All right, mister.')
('00:35:20', '00:35:22', "I'll come along with you.")
('00:35:22', '00:35:23', 'Easy matey, easy.')
('00:35:23', '00:35:25', 'Follow me.')
('00:35:25', '00:35:27', 'Please governor.')
('00:35:29', '00:35:31', "I hadn't had nothing to eat for two days.")


('00:41:33', '00:41:35', 'All right get on with it.')
('00:41:35', '00:41:36', 'Thank you governor.')
('00:41:36', '00:41:37', 'Come on, now.')
('00:41:41', '00:41:43', 'Easy does it.')
('00:41:43', '00:41:44', 'Right there.')
('00:41:44', '00:41:47', 'Good night, my lordships.')
('00:41:47', '00:41:50', 'Told you it was a lot of nonsense.')
('00:41:50', '00:41:52', "I don't understand.")
('00:41:52', '00:41:53', 'That thing must weight a ton.')
('00:41:53', '00:41:55', 'Look at that man staggering.')
('00:41:55', '00:41:56', 'Do you know something, doctor.')
('00:41:56', '00:41:58', "A few tools wouldn't take that much energy.")
('00:41:58', '00:42:00', "We've just looked inside.")
('00:42:00', '00:42:01', 'On the top only.')
('00:42:01', '00:42:02', 'There might be a false bottom.')
('00:42:02', '00:42:03', 'Hey you, you stop there.')
('00:42:03', '00:42:05', "Stop or I'll shoot.")
('00:42:05', '00:42:07', 'Great Scott, Holmes.')
('00:42:22', '00:42:23', "Well I'll be blowed.")
('00:

('00:46:53', '00:46:55', 'without the other three')
('00:46:55', '00:46:56', 'and undoubtedly none of these scientists')
('00:46:56', '00:46:58', 'is known to each other.')
('00:46:58', '00:47:00', 'Professor Moriarity also has the code')
('00:47:00', '00:47:03', 'and we must allow for his ability to decipher it.')
('00:47:03', '00:47:06', "We haven't time to break the fourth code now.")
('00:47:06', '00:47:09', 'We must get to the first three men')
('00:47:09', '00:47:10', 'before Moriarity does.')
('00:47:10', '00:47:12', 'Palace Crescent first?')
('00:47:12', '00:47:13', 'Right.')
('00:47:13', '00:47:14', "I'll go and get a taxi.")
('00:47:14', '00:47:15', 'Miss Eberli,')
('00:47:15', '00:47:16', 'will you please wait here till we return?')
('00:47:16', '00:47:17', 'Thank you.')
('00:47:18', '00:47:19', 'Yeah, where are you going, sir?')
('00:47:26', '00:47:27', 'Jacob Durrei live here?')
('00:47:27', '00:47:28', "Yes, but there's been a bit of trouble")
('00:47:29', '00:47:30', "so

('00:52:10', '00:52:18', 'H-o-f-f-n-e-r,')
('00:52:18', '00:52:25', 'S-l-o-a-n-e.')
('00:52:25', '00:52:30', 'Frederick Hoffner, Sloane Square.')
('00:52:30', '00:52:34', 'Of course.')
('00:52:34', '00:52:36', 'Hoffner would be the perfect selection.')
('00:52:36', '00:52:38', 'Then you want us to tend to Hoffner?')
('00:52:38', '00:52:40', 'No, we can use Hoffner.')
('00:52:40', '00:52:42', "He's a brilliant scientist.")
('00:52:42', '00:52:43', "If Dr. Tobel doesn't recover from your persuasion,")
('00:52:43', '00:52:46', 'Hoffner would be able')
('00:52:46', '00:52:48', 'to put the four parts together.')
('00:52:48', '00:52:49', 'You and Godfrey will call on Hoffner')
('00:52:49', '00:52:51', 'and you will bring him here')
('00:52:51', '00:52:52', 'with the fourth section of the bomb sight.')
('00:52:52', '00:52:54', 'Hurry.')
('00:52:54', '00:52:56', 'Put up your hands, Professor Hoffner.')
('00:53:15', '00:53:18', "I'll take the box.")
('00:53:56', '00:53:58', 'We Meet again, Prof

('00:59:05', '00:59:06', 'injured in alterations with Scotland Yard.')
('00:59:06', '00:59:11', 'I have a fully equipped hospital here.')
('00:59:11', '00:59:14', 'The trail goes this way, inspector.')
('00:59:32', '00:59:35', 'Drop by drop, Holmes.')
('00:59:43', '00:59:45', 'Drop by drop.')
('00:59:45', '00:59:49', "Ahh, in a way I'm almost sorry.")
('00:59:49', '00:59:53', 'You were a stimulating influence to me')
('00:59:53', '00:59:57', 'but it was obvious that I should win in the end.')
('00:59:57', '01:00:00', 'Only a matter of moments now.')
('01:00:04', '01:00:05', 'Take Tobel down to the boat.')
('01:00:06', '01:00:07', 'Start the engine.')
('01:00:07', '01:00:09', 'Closer to the end Holmes,')
('01:00:18', '01:00:21', 'closer and closer.')
('01:00:21', '01:00:24', 'Each second a few more drops leave your')
('01:00:24', '01:00:28', 'desiccated body')
('01:00:28', '01:00:30', 'ending,')
('01:00:30', '01:00:32', 'you can feel me touch you.')
('01:00:32', '01:00:35', "You're perf

=== is number, ~~~ is text, --- is time index. 

=====94

----01:03:40,923 --> 01:03:42,483

~~~~I have a trap set,

~~~~

=====1395

----01:03:42,563 --> 01:03:44,163

~~~~electric eye principle.

~~~~

=====1396

----01:03:44,203 --> 01:03:46,243

~~~~My passing through

~~~~will break the beam

~~~~

=====1397

----01:03:46,283 --> 01:03:48,083

~~~~and automatically open

~~~~the highly deceptive

~~~~

=====1398

----01:03:48,163 --> 01:03:51,723

~~~~trap door behind

~~~~me, of course.

~~~~

=====1399

----01:03:51,803 --> 01:03:54,643

~~~~My pursuer, meaning

~~~~you my dear Holmes,

~~~~

=====1400

----01:03:54,723 --> 01:03:56,003

~~~~will then be plunged

~~~~

=====1401

----01:03:56,043 --> 01:03:57,683

~~~~sixty feet into

~~~~the sewers below.

~~~~

=====1402

----01:03:57,763 --> 01:03:59,083

~~~~Holmes.

~~~~

=====1403

----01:04:04,963 --> 01:04:07,403

~~~~Holmes.

~~~~

=====1404

----01:04:07,483 --> 01:04:09,443

~~~~Poor Moriarity.

~~~~

=====1405

----0

In [43]:
print(test[0:900])

1
00:01:59,443 --> 00:02:02,363
Shh, he's here.

2
00:02:15,803 --> 00:02:17,323
You gentlemen look like

3
00:02:17,363 --> 00:02:18,963
educated men
of good taste.

4
00:02:19,043 --> 00:02:22,443
I have here some rare
old first editions.

5
00:02:22,483 --> 00:02:23,803
He's not bothering us.

6
00:02:23,883 --> 00:02:26,283
I'm interested
in his books.

7
00:02:26,323 --> 00:02:28,283
Okay.

8
00:02:28,363 --> 00:02:32,843
Now here are the
complete works

9
00:02:32,923 --> 00:02:34,723
of William Shakespeare,

10
00:02:34,803 --> 00:02:37,243
an old German writer.

11
00:02:37,323 --> 00:02:38,643
I like that.

12
00:02:38,683 --> 00:02:40,003
We move tonight.

13
00:02:40,083 --> 00:02:41,483
Orders from Berlin.

14
00:02:41,523 --> 00:02:42,803
Dr. Tobel is to be

15
00:02:42,883 --> 00:02:44,363
across the border
befo


In [33]:
print(re.sub('^[^(1)]*','',test)) #removes stuff before the 1

1
00:00:25,258 --> 00:00:27,226
"BuiId your own boat.

2
00:00:27,260 --> 00:00:29,125
All you do
is put it together."

3
00:00:29,162 --> 00:00:31,221
"Even a chiId can do it."

4
00:00:33,466 --> 00:00:35,661
Oh, boy!
Isn't she a beauty?

5
00:00:35,702 --> 00:00:36,760
Ain't she a honey?

6
00:00:36,803 --> 00:00:39,203
Oh, boy!
She's hot stuff.

7
00:00:39,239 --> 00:00:41,969
She's a humdinger.

8
00:00:42,008 --> 00:00:43,999
We'll call her
the 'Queen Minnie.'

9
00:00:51,217 --> 00:00:53,208
Hey! Look out below!

10
00:00:58,391 --> 00:00:59,983
Lend a hand, me hearties.

11
00:01:00,026 --> 00:01:02,085
Aye, aye, captain.

12
00:01:04,097 --> 00:01:05,997
Heave! Ho!

13
00:01:06,032 --> 00:01:07,966
Heave! Ho!

14
00:01:08,001 --> 00:01:08,968
Heave!

15
00:01:09,002 --> 00:01:10,060
Hey! Look out!

16
00:01:17,677 --> 00:01:19,474
All you do
is put it together.

17
00:01:20,680 --> 00:01:23,308
Yeah.
Even a child can do it.

18
00:01:59,319 --> 00:02:02,914
When the bell
in th

In [70]:
def download_sub(best_sub_id):
    if best_sub_id:
        #best_sub_id='1952195191'
        down_data=xmlrpc.DownloadSubtitles(token, [best_sub_id])
        if down_data['status'] !='200 OK':
            print('--------------------- Error Message:')
            print(down_data['status'])
            exit("Exiting Program Now. Come back Tomorrow. ")
        return down_data
    else:
        return False
    
def decode_parse(down_data,imdb_id):
    if down_data:
        compressed_subdata = decode(bytearray(down_data['data'][0]['data'].encode("utf-8")),'base64')
        sub_text = gzip.GzipFile(fileobj=io.BytesIO(compressed_subdata)).read()
        #parse it and store it into a DataFrame with three columns, starttime, endtime, and line
        sub_text_str = str(sub_text)
        snew = sub_text_str.replace("\\n","\n").replace("\\r","\r").replace("</i>","").replace("<i>-","").replace("<i>","").replace("\\","")
        subs=list(filter(None, snew.split('\r\n')))
        subs=[x for x in subs if not x.isdigit()]
        if subs[0][0]=='b':
            subs.pop(0)

        starttimes=[]
        endtimes=[]
        words=[]
        #"-->" in line
        prevLineWasWords =False
        for line in subs:
            line=line.strip()
            if "-->" in line:
                starttimes.append(line[:8])
                endtimes.append(line[17:25])
            elif len(line) > 1:
                    if prevLineWasWords:
                            words[len(words)-1] = words[len(words)-1] + " " + line
                    else:
                            words.append(line)
                    prevLineWasWords = True
                    continue
            prevLineWasWords = False  

        #this needs to be made better but for now we need these to be same length so it will have to do
        trim_by=min([len(starttimes),len(endtimes),len(words)])
        starttimes=starttimes[0:trim_by]
        endtimes=endtimes[0:trim_by]
        words=words[0:trim_by]

        imdb_idi=[imdb_id]
        linfo=['StartTime','EndTime','Line']
        index=pd.MultiIndex.from_product([imdb_idi,linfo],names=['IDs','LData'])
        plines=pd.DataFrame(columns=index)


        s1 = pd.Series(starttimes)
        s2 = pd.Series(endtimes)
        s3 = pd.Series(words)
        plines.loc[:,(imdb_id,'StartTime')] = s1.values
        plines.loc[:,(imdb_id,'EndTime')] = s2.values
        plines.loc[:,(imdb_id,'Line')] = s3.values
        return plines
    else:
        return False

#def populate_db(list):
    #input is a list of IMDB_IDS
    #open existing database of subtitles
    #check if each id is in there
        #if not download it and add it to the list (do this 20 at a time)
        #if yes then move on

In [16]:
##open existing database and download shit from it
if os.path.isfile('movbaseB.h5'):
    movdb=pd.read_hdf('movbaseB.h5', 'imdb_id')
else:
    print('Cannot find file movbaseB!')

In [10]:
batch=movdb[1000:1350]
len(batch)

350

In [11]:
#run this PER GENRE
#input is a list of imdb_id's which can come from whatever
#probably should come from a subset of movdb
if os.path.isfile('Sub_data.h5'):
    Sub_data=pd.read_hdf('Sub_data.h5', 'subs')
else:
    print('Cannot find file Sub_data! Making new one.')

for item in batch['imdb_id']:
    if item not in Sub_data.columns.levels[0]:
            one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),1)),item)
            try:
                if not one_sub:
                    print('COULDNT FIND SUBS FOR {}.   Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
                    continue
            except:
                if len(one_sub) < 2:
                    one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),2)),item)

                    if len(one_sub) < 2:
                        one_sub=decode_parse(download_sub(find_best_file(search_imdbid(item),3)),item)

                        if len(one_sub) <2:
                            print('First 3 subs for {} were bad. Not going any further. Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
                            continue



                Sub_data=pd.concat([Sub_data,one_sub], axis=1)
                Sub_data=Sub_data.fillna('')
                Sub_data.to_hdf('Sub_data.h5', 'subs', mode='w')
                print('Downloaded subs for {}.   Title:  {} '.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))

    else:
        print('Encountered double of {}. Title:  {}'.format(item,batch[batch['imdb_id'] == item]['Title'].values[0]))
        continue
        
print('Done. Writing Sub_data to file.')

Downloaded subs for 3387542.   Title:  The Forest 
Downloaded subs for 0130018.   Title:  I Still Know What You Did Last Summer 
Downloaded subs for 1311067.   Title:  Halloween II 
Downloaded subs for 0433386.   Title:  The Grudge 2 
Downloaded subs for 0099528.   Title:  The Exorcist III 
Downloaded subs for 0097368.   Title:  The Fly II 
Downloaded subs for 0450345.   Title:  The Wicker Man 
Downloaded subs for 0091419.   Title:  Little Shop of Horrors 
Downloaded subs for 1053859.   Title:  The Grudge 3 
Downloaded subs for 0192731.   Title:  Urban Legends: Final Cut 
Downloaded subs for 0220506.   Title:  Halloween: Resurrection 
Downloaded subs for 0475944.   Title:  The Covenant 
Downloaded subs for 0105217.   Title:  Raising Cain 
Downloaded subs for 0155975.   Title:  Psycho 
Downloaded subs for 1270761.   Title:  Don't Be Afraid of the Dark 
Downloaded subs for 0083972.   Title:  Friday the 13th Part III 
Downloaded subs for 0096734.   Title:  The 'Burbs 
Downloaded subs for 

COULDNT FIND SUBS FOR .   Title:  The Story Of Short Stack 
Downloaded subs for 0098213.   Title:  Roger & Me 
Downloaded subs for 1664894.   Title:  Cave of Forgotten Dreams 
Downloaded subs for 0226648.   Title:  Wolves 
Downloaded subs for 0088178.   Title:  Stop Making Sense 
Downloaded subs for 1587707.   Title:  Exit Through the Gift Shop 
Downloaded subs for 0379225.   Title:  The Corporation 
Downloaded subs for 1529567.   Title:  Sea Rex 3D: Journey to a Prehistoric World 
COULDNT FIND SUBS FOR 4266692.   Title:  Drain the Bermuda Triangle 
Downloaded subs for 0100332.   Title:  Paris is Burning 
Downloaded subs for 0388789.   Title:  Born into Brothels 
COULDNT FIND SUBS FOR 3635920.   Title:  D-Day: Normandy 1944 
Downloaded subs for 0109508.   Title:  Crumb 
Downloaded subs for 0389326.   Title:  Riding Giants 
Downloaded subs for 3882074.   Title:  The Eagle Huntress 
Downloaded subs for 0342172.   Title:  Capturing the Friedmans 
Downloaded subs for 1584016.   Title:  Cat

Downloaded subs for 1322313.   Title:  Sunshine Superman 
Downloaded subs for 1112115.   Title:  King Corn 
COULDNT FIND SUBS FOR 0216731.   Title:  Edgeplay: A Film About The Runaways 
Downloaded subs for 1990352.   Title:  The Island President 
Downloaded subs for 0970521.   Title:  Turtle: The Incredible Journey 
COULDNT FIND SUBS FOR 1541666.   Title:  Circo 
COULDNT FIND SUBS FOR 1746153.   Title:  Crime After Crime 
COULDNT FIND SUBS FOR 0430916.   Title:  Beautiful Losers 
COULDNT FIND SUBS FOR .   Title:  The Story Of Short Stack 
COULDNT FIND SUBS FOR 2007995.   Title:  Screaming in High Heels: The Rise & Fall of th.. 
Downloaded subs for 3268458.   Title:  The Internet's Own Boy: The Story of Aaron Swartz 
COULDNT FIND SUBS FOR 0264802.   Title:  Live Nude Girls Unite! 
Downloaded subs for 3270538.   Title:  Requiem for the American Dream 
COULDNT FIND SUBS FOR 1276943.   Title:  Bringing Godzilla Down to Size 
COULDNT FIND SUBS FOR 2234315.   Title:  Occupy Unmasked 
COULDNT

In [12]:
read_data=pd.read_hdf('Sub_data.h5', 'subs')
len(read_data.columns.levels[0])

1601